In [2]:
import pandas as pd
import os
from datetime import datetime
from PIL import Image
from datetime import datetime, timedelta
import time as time_lib
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
TIFF_FOLDER = '/Users/khunnoot/Desktop/year4/senior_project/solarmap_project/Data/cloud_mask'
PLANT_FOLDER = '/Users/khunnoot/Desktop/year4/senior_project/solarmap_project/clear-sky-detection/'
TRASH_FOLDER = '/Users/khunnoot/Desktop/year4/senior_project/solarmap_project/Data/trash_'

In [4]:
#the plant data can map to the coordinate of images

plant_df = pd.read_csv(os.path.join(PLANT_FOLDER,'plant_metadata.csv'))
plant_df.head(5)

,Unnamed: 0,site_name,location_id,site_capacity,point_p,point_i,lat,lng,tile_id,alt,row,col
0,0,ISL001,1,875.0,1279.0,1280.0,14.005230,100.519403,13220313233310,6.0,832,863
1,1,ISL002,9,175.0,1281.0,1282.0,13.706500,100.281200,13220331012031,9.0,849,850
2,2,ISL003,11,875.0,1283.0,1284.0,13.653800,100.522300,13220331013310,5.0,852,864
3,3,ISL004,12,375.0,1285.0,1286.0,13.635425,100.632181,13220331102213,10.0,853,870
4,4,ISL005,13,875.0,1287.0,1288.0,13.692800,100.897200,13220331103131,3.0,849,884


In [34]:
#the satellite images folder

cloud_list = os.listdir(TIFF_FOLDER)
len(cloud_list)

10138

In [ ]:
## clean cloud images ##
trash1 = []
trash2 = []
for file in cloud_list:
    if file.endswith('.tif') or file.endswith('.TIF'):
        I = Image.open(os.path.join(TIFF_FOLDER,file))
        imarray = np.array(I)
        imarray = imarray[:,:,:3]
        sum_row = np.max(imarray,axis=1)
        sum_col = np.max(imarray,axis=0)
        sum_row = np.max(sum_row,axis=1)
        sum_col = np.max(sum_col,axis=1)
    
    if len(sum_row[sum_row==0])>1:
        trash1.append(file)
    if len(sum_col[sum_col==0])>1:
        trash2.append(file)

In [ ]:
# check number of defect folder
q = set(trash2)
q.update(set(trash1))
len(q)

In [ ]:
# change folder of defect images
for file in trash1:
    os.rename(os.path.join(TIFF_FOLDER,file), os.path.join(TRASH_FOLDER,file))
for file in trash2:
    if file not in trash1:
        os.rename(os.path.join(TIFF_FOLDER,file), os.path.join(TRASH_FOLDER,file))

In [ ]:
cloud_list = os.listdir(TIFF_FOLDER)
len(cloud_list)

## Create the images data at Synergy Technology Co., Ltd.

In [5]:
#the META-data that mapping the satellite image and the coordinate with using grid zoom level 14

METADATA = '/Users/khunnoot/Desktop/year4/senior_project/gitSolarmap/CUEESolarmap-main/training/data/metadata'
grid14 = 'grid14_tiff_index_v2.csv'
grid_df = pd.read_csv(os.path.join(METADATA, grid14))

In [32]:
#latitude and longitude of Synergy Technology Co., Ltd.
lat = 14.00294247371573
lng = 100.6267303787081

#find point of Synergy site in row and column in images and the nearest lat,long with using grid zoom level 14
near_lat = abs(grid_df.Lat - lat).idxmin()
row = grid_df.row.loc[near_lat]
d = grid_df[grid_df.row == row]
t = abs(d.Lon - lng)
Synergy = grid_df.loc[[t.idxmin()]]
Synergy['site_name'] = 'Synergy'
Synergy 

,tile_id,Lat,Lon,row,col,site_name
118881,13220313322211,13.99804,100.62378,832,869,Synergy


In [35]:
## map cloud and site position lat/long 5x5
data = []
k=0; ch = 0
for file_name in cloud_list:
    if file_name.lower().endswith('.tif'):
        try:
            I = Image.open(os.path.join(TIFF_FOLDER, file_name))
            imarray = np.array(I)
            time = datetime.strptime(file_name.split('_')[-1].split(".")[0], '%Y%m%d%H%M') + timedelta(hours=7)
            k+=1
            print(k, sep=' ', end='\r', flush=True)
            
            name = 'Synergy'
            row = Synergy['row']
            col = Synergy['col']
            data.append({'site_name': name, 'Datetime': time, \
                         'CI': float(imarray[int(row), int(col), ch])/255,\
                        'CI_1': float(imarray[int(row)-2, int(col)-2, ch])/255,\
                        'CI_2': float(imarray[int(row)-2, int(col)-1, ch])/255,\
                        'CI_3': float(imarray[int(row)-2, int(col), ch])/255,\
                        'CI_4': float(imarray[int(row)-2, int(col)+1, ch])/255,\
                        'CI_5': float(imarray[int(row)-2, int(col)+2, ch])/255,\
                        'CI_6': float(imarray[int(row)-1, int(col)-2, ch])/255,\
                        'CI_7': float(imarray[int(row)-1, int(col)-1, ch])/255,\
                        'CI_8': float(imarray[int(row)-1, int(col), ch])/255,\
                        'CI_9': float(imarray[int(row)-1, int(col)+1, ch])/255,\
                        'CI_10': float(imarray[int(row)-1, int(col)+2, ch])/255,\
                        'CI_11': float(imarray[int(row), int(col)-2, ch])/255,\
                        'CI_12': float(imarray[int(row), int(col)-1, ch])/255,\
                        'CI_14': float(imarray[int(row), int(col)+1, ch])/255,\
                        'CI_15': float(imarray[int(row), int(col)+2, ch])/255,\
                        'CI_16': float(imarray[int(row)+1, int(col)-2, ch])/255,\
                        'CI_17': float(imarray[int(row)+1, int(col)-1, ch])/255,\
                        'CI_18': float(imarray[int(row)+1, int(col), ch])/255,\
                        'CI_19': float(imarray[int(row)+1, int(col)+1, ch])/255,\
                        'CI_20': float(imarray[int(row)+1, int(col)+2, ch])/255,\
                        'CI_21': float(imarray[int(row)+2, int(col)-2, ch])/255,\
                        'CI_22': float(imarray[int(row)+2, int(col)-1, ch])/255,\
                        'CI_23': float(imarray[int(row)+2, int(col), ch])/255,\
                        'CI_24': float(imarray[int(row)+2, int(col)+1, ch])/255,\
                        'CI_25': float(imarray[int(row)+2, int(col)+2, ch])/255})
        except Exception as e:
            print(f"Error processing image {file_name}: {e}")    

In [36]:
cidf = pd.DataFrame(data)
cidf

,site_name,Datetime,CI,CI_1,CI_2,CI_3,CI_4,CI_5,CI_6,CI_7,...,CI_16,CI_17,CI_18,CI_19,CI_20,CI_21,CI_22,CI_23,CI_24,CI_25
0,Synergy,2023-03-08 08:20:00,0.023529,0.031373,0.027451,0.031373,0.023529,0.023529,0.027451,0.027451,...,0.027451,0.023529,0.023529,0.019608,0.015686,0.019608,0.019608,0.019608,0.015686,0.007843
1,Synergy,2023-06-24 09:10:00,0.360784,0.215686,0.278431,0.360784,0.364706,0.333333,0.223529,0.262745,...,0.274510,0.278431,0.329412,0.337255,0.349020,0.321569,0.282353,0.329412,0.360784,0.376471
2,Synergy,2023-03-14 10:10:00,0.415686,0.513725,0.376471,0.337255,0.772549,0.635294,0.545098,0.545098,...,0.615686,0.615686,0.470588,0.670588,0.568627,0.525490,0.666667,0.525490,0.521569,0.584314
3,Synergy,2023-05-24 14:40:00,0.035294,0.000000,0.039216,0.066667,0.000000,0.000000,0.011765,0.015686,...,0.023529,0.027451,0.027451,0.000000,0.000000,0.019608,0.027451,0.023529,0.003922,0.000000
4,Synergy,2023-04-21 18:40:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10132,Synergy,2023-06-29 18:20:00,0.890196,0.811765,0.807843,0.784314,0.898039,0.894118,0.819608,0.854902,...,0.874510,0.890196,0.925490,0.929412,0.933333,0.917647,0.909804,0.933333,0.941176,0.949020
10133,Synergy,2023-04-30 07:40:00,0.337255,0.278431,0.298039,0.352941,0.301961,0.298039,0.231373,0.243137,...,0.215686,0.227451,0.305882,0.274510,0.313725,0.254902,0.203922,0.270588,0.294118,0.309804
10134,Synergy,2023-06-13 07:30:00,0.125490,0.058824,0.098039,0.200000,0.164706,0.145098,0.062745,0.074510,...,0.086275,0.090196,0.086275,0.109804,0.101961,0.070588,0.086275,0.074510,0.078431,0.086275
10135,Synergy,2023-02-08 09:10:00,0.007843,0.007843,0.007843,0.011765,0.000000,0.000000,0.003922,0.003922,...,0.003922,0.003922,0.003922,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [38]:
# cidf.to_csv('ci_cloudmask_Synergy.csv', index = False)

In [39]:
col_name = ['site_name','Datetime']
for d in range(256):
    col_name.append(d)

In [40]:
## map cloud and site position lat/long 16x16
data = []
k=0
    
for file_name in cloud_list:
    if file_name.lower().endswith('.tif'):
        try:
            I = Image.open(os.path.join(TIFF_FOLDER, file_name))
            imarray = np.array(I)
            time = datetime.strptime(file_name.split('_')[-1].split(".")[0], '%Y%m%d%H%M') + timedelta(hours=7)
            k+=1
            print(k, sep=' ', end='\r', flush=True)
            name = 'Synergy'
            row = Synergy['row']
            col = Synergy['col']
            cropped_img1 = imarray[int(row)-8:int(row)+8,int(col)-8:int(col)+8,0]
            pix1 = cropped_img1.reshape(1,256)[0].tolist()
            d = [name,time] + pix1
            data.append(d)
        except Exception as e:
            print(f"Error processing image {file_name}: {e}")

In [41]:
img_df = pd.DataFrame(data,columns=col_name)

In [43]:
img_df.set_index('Datetime', inplace=True)
img_df.index = img_df.index.tz_localize('Asia/Bangkok')
img_df.iloc[:,1:] = img_df.iloc[:,1:]/255

In [44]:
img_df

,site_name,0,1,2,3,4,5,6,7,8,...,246,247,248,249,250,251,252,253,254,255
Datetime,,,,,,,,,,,,,,,,,,,,,
2023-03-08 08:20:00+07:00,Synergy,0.031373,0.031373,0.031373,0.031373,0.031373,0.031373,0.031373,0.031373,0.031373,...,0.003922,0.011765,0.000000,0.003922,0.019608,0.000000,0.003922,0.027451,0.019608,0.019608
2023-06-24 09:10:00+07:00,Synergy,0.376471,0.341176,0.325490,0.325490,0.305882,0.282353,0.301961,0.286275,0.309804,...,0.388235,0.431373,0.447059,0.380392,0.384314,0.376471,0.368627,0.392157,0.419608,0.384314
2023-03-14 10:10:00+07:00,Synergy,0.109804,0.105882,0.109804,0.125490,0.105882,0.074510,0.094118,0.145098,0.090196,...,0.454902,0.474510,0.403922,0.313725,0.384314,0.450980,0.400000,0.396078,0.517647,0.364706
2023-05-24 14:40:00+07:00,Synergy,0.066667,0.000000,0.015686,0.074510,0.000000,0.000000,0.086275,0.000000,0.000000,...,0.019608,0.000000,0.000000,0.000000,0.043137,0.007843,0.070588,0.078431,0.133333,0.227451
2023-04-21 18:40:00+07:00,Synergy,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-29 18:20:00+07:00,Synergy,0.807843,0.925490,0.901961,0.803922,0.960784,0.949020,0.792157,0.925490,0.854902,...,0.901961,0.968627,0.898039,0.886275,0.937255,0.909804,0.905882,0.933333,0.925490,0.901961
2023-04-30 07:40:00+07:00,Synergy,0.364706,0.211765,0.223529,0.352941,0.411765,0.403922,0.345098,0.396078,0.403922,...,0.301961,0.309804,0.274510,0.262745,0.313725,0.462745,0.439216,0.286275,0.403922,0.407843
2023-06-13 07:30:00+07:00,Synergy,0.215686,0.145098,0.152941,0.196078,0.258824,0.239216,0.176471,0.160784,0.156863,...,0.054902,0.062745,0.027451,0.043137,0.062745,0.047059,0.047059,0.050980,0.000000,0.015686


In [45]:
img_df.to_csv('image_cloudmask_Synergy.csv') # export to csv